# Train a Classifier to assign causal genes (UKB NMR 500K GWAS )

In [ ]:
# imports
import numpy as np
import pandas as pd

In [ ]:
# Load datasets
classifier_dataset = pd.read_csv(
    "classifier_dataset.tsv",
    sep="\t",
)
subset_classifier_cholesterol = pd.read_csv(
    "classifier_dataset_cholesterol.tsv",
    sep="\t",
)
subset_classifier_amino_acid = pd.read_csv(
    "classifier_dataset_amino_acid.tsv",
    sep="\t",
)
subset_classifier_lipid = pd.read_csv(
    "classifier_dataset_lipid.tsv",
    sep="\t",
)

## Preprocess datasets 
- Filter variant gene pairs
- remove overlapping variants

In [ ]:
# copy datasets
cholesterol_dataset = subset_classifier_cholesterol.copy()
amino_acid_dataset = subset_classifier_amino_acid.copy()
lipid_dataset = subset_classifier_lipid.copy()

In [ ]:
# remove converging splits between the three datasets

# rsids in cholesterol dataset 547
ids_cholesterol_subset = cholesterol_dataset["variant_id"].unique()

# rsids in amino acid dataset 696
ids_amino_acid_subset = amino_acid_dataset["variant_id"].unique()

# rsids in lipid dataset 704
ids_lipid_subset = lipid_dataset["variant_id"].unique()


# goal ~ 775 per id,
# overlap
overlap_cholesterol_lipid = (
    pd.Series(np.intersect1d(ids_cholesterol_subset, ids_lipid_subset))
    .sample(frac=1)
    .reset_index(drop=True)
)

# Split the sampled ids into two groups
rsid_overlap_for_lipids = overlap_cholesterol_lipid.iloc[:71]
rsid_overlap_for_cholesterol = overlap_cholesterol_lipid.iloc[71:]

# divide
cholesterol_dataset = cholesterol_dataset[
    ~cholesterol_dataset["variant_id"].isin(rsid_overlap_for_lipids)
]
lipid_dataset = lipid_dataset[
    ~lipid_dataset["variant_id"].isin(rsid_overlap_for_cholesterol)
]

# check that no overlap remains
common_rsIDs = np.intersect1d(
    cholesterol_dataset["variant_id"], lipid_dataset["variant_id"]
)
assert len(common_rsIDs) == 0, "There are overlapping variant_ids in the datasets"

## now amino acids: just drop all rsids which are overlapping between the two datasets
amino_acid_dataset = amino_acid_dataset[
    ~amino_acid_dataset["variant_id"].isin(
        ids_cholesterol_subset.tolist() + ids_lipid_subset.tolist()
    )
]

# Step 1: Find unique IDs in the first two datasets
unique_ids_first_two = np.union1d(
    cholesterol_dataset["variant_id"], lipid_dataset["variant_id"]
)

# Step 2: Find common IDs between the unique IDs of the first two and the third dataset
common_ids_all = np.intersect1d(unique_ids_first_two, amino_acid_dataset["variant_id"])

# Step 3: Assert no common IDs exist
assert len(common_ids_all) == 0, "There are overlapping variant_ids among the datasets"

### Encode Columns

In [ ]:
from sklearn.preprocessing import LabelEncoder


def encode_columns(dataset, columns, pre_existing_encoders=None):
    """
    Encodes specified columns of a dataset using LabelEncoder, with an option to use pre-existing encoders.

    Parameters:
    - dataset: pandas DataFrame to be modified.
    - columns: List of column names in the dataset to be encoded.
    - pre_existing_encoders: Optional; Dictionary of pre-existing LabelEncoders for specified columns.

    Returns:
    - Modified dataset with specified columns encoded.
    - Dictionary of used encoders for the specified columns.

    """
    if pre_existing_encoders is None:
        pre_existing_encoders = {}  # Initialize if no encoders are provided

    encoders = pre_existing_encoders.copy()  # Copy to avoid modifying the original dict
    dataset_copy = dataset.copy()
    for column in columns:
        if column in encoders:
            # Use the pre-existing encoder for this column
            encoder = encoders[column]
            dataset_copy[column] = encoder.transform(dataset_copy[column])
        else:
            # Create a new encoder, fit it, and transform the data
            encoder = LabelEncoder()
            dataset_copy[column] = encoder.fit_transform(dataset_copy[column])
            encoders[column] = encoder  # Store the new encoder

    return dataset_copy, encoders


# Use label encoding:
# encode the classifier dataset first to use this encoder on all subsets
columns_to_encode = [
    "ensembl_id",
    "variant_id",
    "gene_biotype",
    "hgnc_symbol",
    "consequence_lead",
    "consequence_proxy",
    "IMPACT_lead",
    "IMPACT_proxy",
]
classifier_dataset_encoded, classifier_dataset_encoders = encode_columns(
    classifier_dataset, columns_to_encode
)
# apply existing encoder on all subsets
cholesterol_dataset, cholesterol_encoders = encode_columns(
    cholesterol_dataset,
    columns_to_encode,
    pre_existing_encoders=classifier_dataset_encoders,
)
amino_acid_dataset, amino_acid_encoders = encode_columns(
    amino_acid_dataset,
    columns_to_encode,
    pre_existing_encoders=classifier_dataset_encoders,
)
lipid_dataset, lipid_encoders = encode_columns(
    lipid_dataset, columns_to_encode, pre_existing_encoders=classifier_dataset_encoders
)

### Train a Random Forest Model

In [ ]:
# imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    auc,
    balanced_accuracy_score,
    cohen_kappa_score,
    confusion_matrix,
    precision_recall_curve,
    roc_curve,
)
from sklearn.model_selection import (
    GridSearchCV,
    cross_val_score,
)

from sklearn.model_selection import GroupKFold, GroupShuffleSplit

In [ ]:
# rf training function
def random_forest_grouped_cv_with_tuning(
    data,
    exclude,
    target,
    cv=5,
    n_jobs=-1,
    random_state=None,
    param_grid=None,
    scoring="balanced_accuracy",
    holdout_size=0.3,
    group_column="variant_id",
):
    # Initial split: training set (for tuning) and holdout set (for final evaluation)
    X, y = data.drop(columns=exclude + [target]), data[target]
    groups = data[group_column]
    gss = GroupShuffleSplit(test_size=0.3, random_state=None)
    for train_index, holdout_index in gss.split(X, y, groups=groups):
        X_train, X_holdout = X.iloc[train_index], X.iloc[holdout_index]
        y_train, y_holdout = y.iloc[train_index], y.iloc[holdout_index]
        groups_train, groups_holdout = (
            groups.iloc[train_index],
            groups.iloc[holdout_index],
        )
        break

    assert set(groups_train).isdisjoint(
        set(groups_holdout)
    ), "Training and holdout groups have overlapping elements."

    # Initialize the Random Forest classifier
    clf = RandomForestClassifier(
        random_state=random_state, class_weight="balanced", n_jobs=n_jobs
    )

    # If no param_grid is provided, use this one
    if param_grid is None:
        param_grid = {
            "max_depth": [80, 90, 100, 110],
            "max_features": [2, 3],
            "min_samples_leaf": [3, 4, 5],
            "min_samples_split": [8, 10, 12],
            "n_estimators": [100, 200, 300, 1000],
        }

    # Initialize GroupKFold
    group_kfold = GroupKFold(n_splits=cv)

    # Initialize GridSearchCV with GroupKFold
    grid_search = GridSearchCV(
        clf, param_grid, cv=group_kfold, n_jobs=n_jobs, scoring=scoring
    )

    # Fit the model on the training data
    grid_search.fit(X_train, y_train.values.ravel(), groups=groups_train)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Make predictions on the holdout set
    predictions = best_model.predict(X_holdout)
    predictions_prob = best_model.predict_proba(X_holdout)
    predictions_prob_greater_05 = np.where(predictions_prob[:, 1] > 0.5, 1, 0).tolist()

    # Calculate metrics
    cm = confusion_matrix(y_holdout, predictions_prob_greater_05)
    fpr, tpr, _ = roc_curve(y_holdout, predictions_prob[:, 1])
    roc_auc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_holdout, predictions_prob[:, 1])
    pr_auc = auc(recall, precision)
    accuracy = accuracy_score(y_holdout, predictions)
    balanced_accuracy = balanced_accuracy_score(y_holdout, predictions)
    cohen_kappa = cohen_kappa_score(y_holdout, predictions)

    # Cross-validated score of the best model
    cv_scores = cross_val_score(
        best_model,
        X_train,
        y_train,
        cv=group_kfold,
        scoring="accuracy",
        groups=groups_train,
    )

    feature_importance = pd.Series(
        best_model.feature_importances_, index=X_train.columns
    )

    # Return a dictionary containing the best model, its parameters, predictions, and metrics
    return {
        "best_model": best_model,
        "best_params": grid_search.best_params_,
        "predictions": predictions,
        "predictions_prob": predictions_prob,
        "predictions_prob_greater_05": predictions_prob_greater_05,
        "confusionMatrix": cm,
        "roc_curve": (fpr, tpr, roc_auc),
        "precision_recall_curve": (precision, recall, pr_auc),
        "accuracy": accuracy,
        "balanced_accuracy": balanced_accuracy,
        "cv_scores": cv_scores,
        "mean_cv_score": np.mean(cv_scores),
        "feature_importance": feature_importance,
        "cohen_kappa": cohen_kappa,
        "roc_auc": roc_auc,
        "y_holdout": y_holdout,
    }

In [ ]:
# train with new param grid
rf_clf_cholesterol = random_forest_grouped_cv_with_tuning(
    cholesterol_dataset,
    exclude=[
        "ensembl_id",
        "variant_id",
        "variant_id.hg19",
        "hgnc_symbol",
        "GO",
        "KEGG",
        "reactome",
        "kegg_and_reactome_pathway_specific_gene",
        "kegg_pathway_specific_gene",
        "reactome_pathway_specific_gene",
        "kegg_or_reactome_pathway_specific_gene",
        "eric_fauman_egea_pair",
        "eric_fauman_egea_gene",
    ],
    target="kegg_or_reactome_pathway_specific_gene",
    param_grid=cholesterol_param_grid,
)

In [ ]:
# training the models for lipid and amino acid datasets
# ...
# ...

## Make predictions 

In [ ]:
# cholesterol
predictions_rf_cholesterol = rf_clf_cholesterol["best_model"].predict_proba(
    classifier_dataset_encoded.drop(
        columns=[
            "ensembl_id",
            "variant_id",
            "variant_id.hg19",
            "hgnc_symbol",
            "GO",
            "KEGG",
            "reactome",
        ]
    )
)
# cholesterol
predictions_rf_lipid = rf_clf_lipid["best_model"].predict_proba(
    classifier_dataset_encoded.drop(
        columns=[
            "ensembl_id",
            "variant_id",
            "variant_id.hg19",
            "hgnc_symbol",
            "GO",
            "KEGG",
            "reactome",
        ]
    )
)
# amino acid
predictions_rf_amino_acid = rf_clf_amino_acid["best_model"].predict_proba(
    classifier_dataset_encoded.drop(
        columns=[
            "ensembl_id",
            "variant_id",
            "variant_id.hg19",
            "hgnc_symbol",
            "GO",
            "KEGG",
            "reactome",
        ]
    )
)


In [ ]:
# add results to the decoded classifier dataset
# take median of prediction scores 

classifier_dataset_w_predictions["prediction_median"] = (
    classifier_dataset_w_predictions[
        [
            "predictions_rf_cholesterol_kegg_reactome_subset_on_cd",
            "predictions_rf_lipid_kegg_reactome_subset_on_cd",
            "predictions_rf_amino_acid_kegg_reactome_subset_on_cd",
        ]
    ].median(axis=1)
)
# label in_training_set ids
classifier_dataset_w_predictions["in_training_set"] = classifier_dataset_w_predictions[
    "variant_id"
].apply(lambda x: 1 if x in all_ids_in_subsets else 0)

idx = classifier_dataset_w_predictions.groupby("variant_id")[
    "prediction_median"
].idxmax()
# get the variant_id gene_pair with the highest median prediction score

idx = classifier_dataset_w_predictions.groupby("variant_id")[
    "prediction_median"
].idxmax()

# Step 2: Create a new column and set it to 1 for these rows
classifier_dataset_w_predictions["prediction_median_pick"] = 0
classifier_dataset_w_predictions.loc[idx, "prediction_median_pick"] = 1


classifier_dataset_w_predictions.to_csv(
    "output/predctions_classifer.tsv",
    sep="\t",
)